# Lagged Features

In [1]:
# TODO: I read in Franzi's merged/final DF as df_full. This name must be replaced by her name in the following cells

In [13]:
import pandas as pd
import numpy as np
df_full = pd.read_parquet("data_s2000_currently.parquet")
df_full = df_full[df_full["shopper"]<2000][["week","shopper","product","product_bought"]]
df_90 = pd.read_parquet("df_90.parquet")
df_full = pd.concat([df_full, df_90], ignore_index=True)

In [6]:
df_full

,week,shopper,product,product_bought
0,0,0,5,0
1,0,0,16,0
2,0,0,27,0
3,0,0,35,0
4,0,0,52,0
...,...,...,...,...
3590887,90,249,225,0
3590888,90,249,227,0
3590889,90,249,231,0
3590890,90,249,234,0


In [3]:
# from module_lags import LagCalculator
import module_lags

In [4]:
purchase_lags = module_lags.LagCalculator(
    df_full[["shopper", "product", "week", "product_bought"]]
    .sort_values(by=["shopper", "product", "week"], ascending=[True, True, True])
    .reset_index(drop=True)
)

In [5]:
lags = purchase_lags.calculate_lags()

0% done.
10% done.
20% done.
30% done.
40% done.
50% done.
60% done.
70% done.
80% done.
90% done.
100% done.


In [8]:
lags.iloc[3001:3006]

,shopper,product,week,product_bought,lag_weeks_of_product_per_customer
3001,1,146,5,0,4
3002,1,146,7,0,6
3003,1,146,8,0,7
3004,1,146,11,1,10
3005,1,146,12,0,1


In [38]:
# replace "product_bought" of week 90 with NaN's
lags.loc[lags['week']==90,['product_bought']] = np.nan

In [41]:
purchase_temporal_distribution = purchase_lags.calculate_purchase_temporal_distribution(lags)
purchase_temporal_distribution.head()

,shopper,product,purchase_temporal_distribution
0,0,4,50.250000
1,0,11,59.666667
2,0,15,10.000000
3,0,16,45.000000
4,0,21,86.000000


In [42]:
avg_no_weeks_between_two_purchases = purchase_lags.calculate_avg_no_weeks_between_two_purchases(lags)
avg_no_weeks_between_two_purchases.head()

,shopper,product,avg_no_weeks_between_two_purchases
0,0,4,5.625000
1,0,11,6.777778
2,0,15,-1.000000
3,0,16,15.500000
4,0,21,-1.000000


In [43]:
# export
lags.to_parquet("lags.parquet")
purchase_temporal_distribution.to_parquet("purchase_temporal_distribution.parquet")
avg_no_weeks_between_two_purchases.to_parquet("avg_no_weeks_between_two_purchases.parquet")

In [44]:
# example
lags[(lags["shopper"]==333) & (lags["week"]==66)]

,shopper,product,week,product_bought,lag_weeks_of_product_per_customer
606545,333,20,66,0.0,-1
606738,333,41,66,0.0,5
606815,333,48,66,0.0,1
606836,333,51,66,0.0,-1
606911,333,60,66,1.0,3
607021,333,80,66,0.0,-1
607302,333,129,66,0.0,8
607341,333,132,66,0.0,6
607401,333,137,66,1.0,3
607500,333,144,66,1.0,12
